<a href="https://colab.research.google.com/github/creamburger/TensorStudy/blob/main/12%EA%B0%95_%EC%9B%8C%EB%93%9C_%EC%9E%84%EB%B2%A0%EB%94%A9_%EA%B0%95%EC%82%AC%EC%9E%90%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

In [ ]:
tfds.disable_progress_bar()

In [ ]:
embedding_layer = layers.Embedding(1000, 5)

In [ ]:
embedding_layer(tf.constant([4, 2, 5, 1]))

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[-0.0363781 , -0.02147125,  0.01166624,  0.02528825,  0.016728  ],
       [-0.01381253,  0.0045158 ,  0.01673272, -0.00044552,  0.03093369],
       [ 0.04909632,  0.02010305,  0.00859509,  0.00967612,  0.04444256],
       [-0.00751492,  0.03140331, -0.0387894 ,  0.01327714, -0.00383195]],
      dtype=float32)>

In [ ]:
# 1) Word Split: "My name is Chris" => ["My", "name", "is", "Chris"]
# 2) Label Encoding: ["My", "name", "is", "Chris"] => [4, 2, 5, 1] 
# 3) Word Embedding: [4, 2, 5, 1] => [[-0.0363781 , -0.02147125,  0.01166624,  0.02528825,  0.016728],...]

In [ ]:
 (train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteOYJLBG/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteOYJLBG/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteOYJLBG/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

In [ ]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

array([[2167,    2,   14, ...,    0,    0,    0],
       [  62,   27,   18, ...,    0,    0,    0],
       [5615,  363, 7514, ...,    0,    0,    0],
       ...,
       [ 909, 6639, 1098, ...,    0,    0,    0],
       [  12,  393,   14, ...,    0,    0,    0],
       [  19,  233, 1942, ...,    0,    0,    0]])

In [ ]:
encoder.vocab_size

8185

In [ ]:
embedding_dim = 16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_batches,
          epochs=10,
          validation_data=test_batches,
          validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 17s 7ms/step - loss: 0.5105 - accuracy: 0.6926 - val_loss: 0.4078 - val_accuracy: 0.8200
Epoch 2/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.2868 - accuracy: 0.8833 - val_loss: 0.3145 - val_accuracy: 0.8700
Epoch 3/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.2273 - accuracy: 0.9107 - val_loss: 0.3755 - val_accuracy: 0.8600
Epoch 4/10
2500/2500 [==============================] - 16s 7ms/step - loss: 0.1946 - accuracy: 0.9249 - val_loss: 0.3229 - val_accuracy: 0.8700
Epoch 5/10
2500/2500 [==============================] - 17s 7ms/step - loss: 0.1734 - accuracy: 0.9367 - val_loss: 0.3128 - val_accuracy: 0.8700
Epoch 6/10
2500/2500 [==============================] - 16s 7ms/step - loss: 0.1525 - accuracy: 0.9442 - val_loss: 0.5495 - val_accuracy: 0.8400
Epoch 7/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.1420 - accuracy: 0.9474 - val_loss: 0.3825 - val_ac